In [ ]:
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import solve_ivp

from google.colab import drive
drive.mount('/path')

# Activation Functions

In [ ]:
# Activation Functions

# Soft Thresholding
def softt(x, thr_def):
    return np.where(np.abs(x) > thr_def, np.sign(x) * (np.abs(x) - thr_def), 0)

# ReLU
def relu(x, thr_def):
    if isinstance(x - thr_def, (int, float)):
        return max(0, x - thr_def)
    else:
        y = np.zeros(x.shape)
        idx = np.where(x - thr_def > 0)
        y[idx] = x[idx]
        return y

# RNN dynamics

In [ ]:
def FCN(t_fcn, x_fcn, F_fcn, Phi_fcn, u_fcn, thr_fcn):
    """
    Firing-rate Competitive Network:
       \dot x = - x + F((I_n - Phi^T*Phi)x + \Phi^T u)
       ARGS:
       -   F_fcn = activation function
       - Phi_fcn = dictionary
       -   u_fcn = input
       - thr_fcn = threshold of activation function
    """
    N_fcn = Phi_fcn.shape[1]
    x_fcn = x_fcn.reshape(N_fcn,1)
    y_fcn = (- x_fcn + F_fcn((np.eye(N_fcn) - Phi_fcn.T @ Phi_fcn) @ x_fcn + (Phi_fcn.T @ u_fcn).reshape(N_fcn,1), thr_fcn)).reshape(N_fcn,)
    return y_fcn

In [ ]:
def LCA(t_lca, x_lca, F_lca, Phi_lca, u_lca, thr_lca):
    """
    LCA (Hopfield-like Neural Network):
       \dot x = - x + (I_n - Phi^T*Phi)F(x) + \Phi^T u
       ARGS:
       -   F_lca = activation function
       - Phi_lca = dictionary
       -   u_lca = input
       - thr_lca = threshold of activation function
    """
    N_lca = Phi_lca.shape[1]
    x_lca = x_lca.reshape(N_lca,1)
    y_lca = - x_lca + (np.eye(N_lca) - Phi_lca.T @ Phi_lca)@F_lca(x_lca,thr_lca) + Phi_lca.T @ u_lca
    return y_lca.reshape(N_lca,)

# Simulations

## Generate Dataset - Parameters

In [ ]:
'''
# Generate Dataset

# M = Input size
# N = No. of vectors in the Dictionary
# s = No. of non-zero entries in the Sparse Input

M = 256
N = 512
s = 5

# Sample s random library entries and associate each of them with a real number.
sampler = random.sample(range(1, N), s)

print(np.sort(sampler))
y0 = np.zeros((N,1))
for k in sampler:
    y0[k] = np.abs(np.random.randn())

# Define the Dictionary
#Balavoine, Romberg, and Rozel 2012 paper:
#- col_1 = first 256 columns, which are the canonical Basis
#- col_2 = next 256 entries, which are the "Sinusoidal Basis"

Phi = np.eye(M,N)
col_1 = np.linspace(0, 1, M)
col_2 = 256
while(col_2 < N):
    Phi[:,col_2] = np.sin((col_2 - 255)  * np.pi * col_1)
    col_2 = col_2 + 1

# Normalize Dictionary
Phi = Phi/ np.sqrt(np.sum(np.square(Phi),0))

# Generate Input
std = 0.0062
eta = std * np.random.randn(M,1)
u = Phi @ y0 + eta

np.savez('/path', M = M, N = N, s = s, sampler = sampler, y0 = y0, Phi = Phi, u = u, std = std, eta = eta)
'''

In [ ]:
# Load the variables from the saved dataset
# add your path
loaded_data = np.load('/path')

# Access the variables using their names
M = int(loaded_data['M'])
N = int(loaded_data['N'])
s = int(loaded_data['s'])
sampler = loaded_data['sampler']
y0 = loaded_data['y0']
Phi = loaded_data['Phi']
u = loaded_data['u']

In [ ]:
# Parameters Simulation
thr = 0.025           # threshold

ti = 0                # initial time
tf = 15               # final time
x0 = np.zeros(N)      # initial condition
x0[5] = 1
x0[15] = 0.5
x0[35] = 0.7
x0[25] = 1
x0[45] = 1.2
x0[55] = 1.35
x0[45] = 0.4
x0[150] = 0.2
x0[350] = 0.3
x0[250] = 1.1
x0[450] = 1.25
x0[57] = 1.45
x0[450] = 0.25
x0[1] = 0.15
x0[3] = 0.8
x0[2] = 1.4
x0[4] = 1
x0[505] = 1.15
x0[405] = 0.44
x0[100] = 0.6

In [ ]:
t_span = (ti, tf)
sol_PFCN = solve_ivp(FCN, t_span, x0, args = (relu, Phi, u, thr), t_eval=np.linspace(t_span[0], t_span[1], 100))

act_func = softt          # activation function
sol_H = solve_ivp(LCA, t_span, x0, args = (act_func, Phi, u, thr), t_eval=np.linspace(t_span[0], t_span[1], 100))

# Visualize the results
t_F = sol_PFCN.t
y_F = sol_PFCN.y             # State evolution FCN

t_H = sol_H.t
y_H = sol_H.y                # State evolution LCA

## FINAL PLOTS

In [ ]:
# Plot N.1 :
# Time evolution of the PFCN and LCA (with soft) state variables

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey = False)
plt.rcParams.update({'font.size': 14})
for i in range(N):
    ax1.plot(t_F, y_F[i])
for k in range(N):
    if y0[k] != 0:
        ax1.plot(tf - 0.2, y0[k], 'rx')
ax1.set_xlabel(r'$t$', fontsize=15)
ax1.set_ylabel('State', fontsize=15)
ax1.set_xlim(left = 0, right = tf)
x_ticks = np.linspace(0, 15, 6)
ax1.set_xticks(x_ticks)
ax1.set_xticklabels([f'{tick:.0f}' if tick != 0 else '0' for tick in x_ticks])
y_ticks = np.linspace(-0.4, 1.6, 11)
ax1.set_yticks(y_ticks)
ax1.set_yticklabels([f'{tick:.1f}' if tick != 0 else '0' for tick in y_ticks])
ax1.grid(True)

for i in range(N):
    ax2.plot(t_H, y_H[i])
for k in range(N):
    if y0[k] != 0:
        ax2.plot(tf - 0.2, y0[k], 'rx')
ax2.set_xlabel(r'$t$', fontsize=15)
ax2.set_xlim(left=0, right = tf)
ax2.set_ylim(bottom = -0.4)
ax2.set_xticks(x_ticks)
ax2.set_xticklabels([f'{tick:.0f}' if tick != 0 else '0' for tick in x_ticks])
ax2.set_yticks(y_ticks)
ax2.set_yticklabels([f'{tick:.1f}' if tick != 0 else '0' for tick in y_ticks])
ax2.grid(True)
plt.show()

In [ ]:
# Plot N.2 :
# Generate and plot PFCN trajectories starting from random initial states
# Specifically, two randomly chosen nodes from the final active (leftward panel) and inactive (rightward panel) set

t_span_plot3 = (ti, 20)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey = False)
for i in range(20):
    x0_rand = np.random.uniform(low = 0.1, high = 1.4, size = N)     # initial condition
    sol_FR = solve_ivp(FCN, t_span_plot3, x0_rand, args = (relu, Phi, u, thr), t_eval=np.linspace(t_span_plot3[0], t_span_plot3[1], 100))
    t_FR = sol_FR.t
    y_FR = sol_FR.y
    ax1.plot(y_FR[sampler[1], 0], y_FR[sampler[2], 0], 'o', markersize = 3)
    ax1.plot(y_FR[sampler[1], -1], y_FR[sampler[2], -1], 'k*', markersize = 5)
    ax1.plot(y_FR[sampler[1],:],y_FR[sampler[2],:], alpha = .8) # alpha is used as a parameter to control the transparency or opacity of the lines when plotting
    ax1.set_xlabel(r'$x_{61}$')
    ax1.set_ylabel(r'$x_{469}$')
    ax1.set_xlim(left = 0, right = 1.8)
    ax1.set_ylim(bottom = 0, top = 1.4)
    ax1.grid(True)

    ax2.plot(y_FR[9, 0], y_FR[99, 0], 'o', markersize = 3)
    ax2.plot(y_FR[9, -1], y_FR[99, -1], 'k*', markersize = 5)
    ax2.plot(y_FR[9,:],y_FR[99,:], alpha = .8)
    ax2.set_xlabel(r'$x_{10}$')
    ax2.set_ylabel(r'$x_{100}$')
    ax2.set_xlim(left = - 0.01, right = 1.8)
    ax2.set_ylim(bottom = - 0.01, top = 1.4)
    ax2.grid(True)
plt.show()

In [ ]:
# Plot N.3 :
# Time evolution of the PFCN and LCA (with ReLu as activation function) state variables

sol_H_relu = solve_ivp(LCA, t_span, x0, args = (relu, Phi, u, thr), t_eval=np.linspace(t_span[0], t_span[1], 100))
t_H_relu = sol_H_relu.t
y_H_relu = sol_H_relu.y

plt.figure(figsize=(6, 5))
plt.rcParams.update({'font.size': 14})
for i in range(N):
    plt.plot(t_H_relu, y_H_relu[i])
for k in range(N):
    if y0[k] != 0:
        plt.plot(tf - 0.2, y0[k], 'rx')
plt.ylabel('State', fontsize=15)
plt.xlabel(r'$t$', fontsize=15)
plt.xlim(left=0, right = tf)
plt.ylim(bottom = -0.42)
plt.grid(True)
plt.show()